<a href="https://colab.research.google.com/github/mgersins-design/Machine-Learning/blob/main/HW7/ConvNN_CIFAR_10_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn

import datetime

In [2]:
transform_process = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2470, 0.2435, 0.2616]
    )
])

data_path = './data'

cifar10_train = datasets.CIFAR10(
    data_path, train=True, download=True, transform=transform_process
)
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True, transform=transform_process
)

batch_size = 256
train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True, num_workers = 2)
val_loader = DataLoader(cifar10_val, batch_size=batch_size, shuffle=False, num_workers = 2)

print(f"Trainingsdaten: {len(cifar10_train)} Bilder")
print(f"Validierungsdaten: {len(cifar10_val)} Bilder")

100%|██████████| 170M/170M [00:03<00:00, 42.8MB/s]


Trainingsdaten: 50000 Bilder
Validierungsdaten: 10000 Bilder


In [3]:
# Creating the model
class Net (nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size = 3, padding = 1)
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(2)

    self.conv2 = nn.Conv2d(16, 8, kernel_size = 3, padding = 1)
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(2)

    self.conv3 = nn.Conv2d(8, 4, kernel_size = 3, padding = 1)
    self.act3 = nn.ReLU()
    self.pool3 = nn.MaxPool2d(2)

    self.fc1 = nn.Linear(4*4*4, 32)
    self.act3 = nn.ReLU()
    self.fc2 = nn.Linear(32, 10)

  def forward(self, x):
    out = self.pool1(self.act1(self.conv1(x)))
    out = self.pool2(self.act2(self.conv2(out)))
    out = self.pool3(self.act3(self.conv3(out)))
    out = out.view(-1, 4*4*4)
    out = self.act3(self.fc1(out))
    out = self.fc2(out)
    return out

In [4]:
# Switching to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")

Training on cuda


In [5]:
def train_and_validate(n_epochs, optimizer, model, loss_fn, train_loader, val_loader):
  device = next(model.parameters()).device

  for epoch in range(1, n_epochs+1):
    # Training Phase
    model.train()
    loss_train = 0.0
    correct_train = 0
    total_train = 0

    for imgs, labels in train_loader:
      imgs = imgs.to(device = device)
      labels = labels.to(device = device)

      outputs = model(imgs)
      loss = loss_fn(outputs, labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_train += loss.item()
      #_, predicted = torch.max(outputs, dim=1)
      #total_train += labels.size(0)
      #correct_train += int((predicted == labels).sum())

    model.eval()
    correct_val = 0
    total_val = 0

    with torch.no_grad():
      for imgs, labels in val_loader:
        imgs = imgs.to(device = device)
        labels = labels.to(device = device)

        outputs = model(imgs)
        _, predicted = torch.max(outputs, dim=1)
        total_val += labels.shape[0]
        correct_val += int((predicted == labels).sum())

    avg_loss = loss_train / len(train_loader)
    #acc_train = correct_train / total_train
    acc_val = correct_val / total_val

    if epoch == 1 or epoch % 10 == 0:
      print('{} Epoch {:3d}, Training loss {:.4f}, Val Acc: {:.2f}'.format(
          datetime.datetime.now(), epoch, avg_loss, acc_val))


In [6]:
model = Net().to(device = device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
loss_fn = nn.CrossEntropyLoss()

train_and_validate(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
    val_loader= val_loader
)

2025-11-25 01:57:44.129040 Epoch   1, Training loss 1.9578, Val Acc: 0.38
2025-11-25 01:58:49.886966 Epoch  10, Training loss 1.2891, Val Acc: 0.53
2025-11-25 02:00:03.837056 Epoch  20, Training loss 1.1545, Val Acc: 0.59
2025-11-25 02:01:17.887813 Epoch  30, Training loss 1.0935, Val Acc: 0.60
2025-11-25 02:02:31.335452 Epoch  40, Training loss 1.0640, Val Acc: 0.60
2025-11-25 02:03:45.453697 Epoch  50, Training loss 1.0322, Val Acc: 0.62
2025-11-25 02:04:58.826004 Epoch  60, Training loss 1.0130, Val Acc: 0.62
2025-11-25 02:06:11.852933 Epoch  70, Training loss 0.9916, Val Acc: 0.63
2025-11-25 02:07:25.214140 Epoch  80, Training loss 0.9781, Val Acc: 0.63
2025-11-25 02:08:38.940481 Epoch  90, Training loss 0.9683, Val Acc: 0.64
2025-11-25 02:09:52.318433 Epoch 100, Training loss 0.9581, Val Acc: 0.63


In [7]:
# Model Complexity
numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(4310, [432, 16, 1152, 8, 288, 4, 2048, 32, 320, 10])